In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/mnist_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-03-22 18:11:09,942 SSD INFO: Loaded configuration file configs/mnist_tdt4265_server.yaml
2021-03-22 18:11:09,944 SSD INFO: 
MODEL:
    NUM_CLASSES: 11
    THRESHOLD: 0.6
    BACKBONE:
        NAME: 'improved'
        PRETRAINED: False
        OUT_CHANNELS: [512, 1024, 512, 512, 256, 256]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES: [[15, 15], [60, 60], [111, 111], [162, 162], [213, 213], [264, 264]]
        MAX_SIZES: [[60, 60], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]
        ASPECT_RATIOS: [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
        BOXES_PER_LOCATION: [4, 6, 6, 6, 4, 4]
INPUT:
    IMAGE_SIZE: [300, 300]
    PIXEL_MEAN: [123.675, 116.280, 103.530]
    PIXEL_STD: [1, 1, 1]
DATASETS:
    TRAIN: ("mnist_detection_train", "mnist_detection_val")
    TEST: ("mnist_detection_val", )
SOLVER:
    MAX_ITER: 15000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 5e-4
OUTPUT_DIR: 'outputs/improved'
DATASET_DIR: "/work/datasets"

2021-03-22 18:11:09,945 SSD INFO: 

In [ ]:
model = start_train(cfg)

Detector initialized. Total Number of params:  36.15M
Backbone number of parameters: 33.94M
SSD Head number of parameters: 2.21M
2021-03-22 18:11:12,106 SSD.trainer INFO: No checkpoint found.
2021-03-22 18:11:13,501 SSD.trainer INFO: Start training ...
2021-03-22 18:11:28,230 SSD.trainer INFO: iter: 000010, lr: 0.00050, total_loss: 24.622 (24.622), reg_loss: 4.043 (4.043), cls_loss: 20.579 (20.579), time: 1.350 (1.350), eta: 5:37:19, mem: 12309M
2021-03-22 18:11:32,507 SSD.trainer INFO: iter: 000020, lr: 0.00050, total_loss: 18.467 (21.544), reg_loss: 2.759 (3.401), cls_loss: 15.708 (18.143), time: 0.428 (0.889), eta: 3:41:56, mem: 12309M
2021-03-22 18:11:36,794 SSD.trainer INFO: iter: 000030, lr: 0.00050, total_loss: 16.059 (19.716), reg_loss: 2.335 (3.046), cls_loss: 13.724 (16.670), time: 0.429 (0.736), eta: 3:03:31, mem: 12309M
2021-03-22 18:11:41,082 SSD.trainer INFO: iter: 000040, lr: 0.00050, total_loss: 13.740 (18.222), reg_loss: 1.877 (2.754), cls_loss: 11.863 (15.468), time: 

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)